# Calibration des probabilités et quelques notions ☕️

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être sensibilisé&nbsp;:
    * aux difficultés à estimer la loi conditionnelle $\eta_k(x)=\mathbb{P}(Y=k|X=x)$.
* Être capable de&nbsp;:
    * de calibrer les probabilités d'un réseau de neurones pour améliorer le fit de $\eta$.



 ----
## I. Introduction

Notez que par soucis de simplicité, nous considérons dans nos formules le cas de la classification binaire. Le propos se généralise bien sûr !

Considérons un problème de classification où $\mathcal{X}$ représente notre espace d'entrée et $\mathcal{Y}=\{0, 1\}$ l'ensemble de nos classes (ici deux classes). Notons $X,Y$ deux variables aléatoires sur $\mathcal{X}\times\mathcal{Y}$ et notons $\mu$ la mesure de $X$ et $\eta(x)=\mathbb{P}(Y=1|X=x)$ la "probabilité *a posteriori*". Notre objectif est assez traditionnellement de trouver une application $h:\mathcal{X}\rightarrow\mathcal{Y}$ telle que le risque suivant est minimisé&nbsp;:

$$R(h)=\mathbb{E}\big[\textbf{1}\{h(X)\neq Y\}\big].$$

Ne connaissant ni $\mu$ ni $\eta$, nous ne pouvons estimer ce risque et devons l'estimer empiriquement en collectant un jeu de données $S_n=\{(X_i, Y_i)\}_{i\leq n}$. Utilisant ce jeu de données, nous pouvons construire la notion de risque empirique&nbsp;:

$$R_n(h)=\frac{1}{n}\sum_{i=1}^n \textbf{1}\{h(X_i)\neq Y_i\}.$$

La séquence sur les fonctions proxy nous a montré que minimiser ce risque est généralement difficile. Nous ne pouvons en particulier pas utiliser la descente de gradient puisque ce dernier est presque partout nul. Nous avons vu que nous pouvions cependant construire une stratégie où notre application ne retourne pas un label mais un score sur $\mathbb{R}$ (qu'on appelle logit en *deep learning* ou en régression logistique), score dont le signe nous permet de déterminer la classe. Une *loss* possédant un certain nombre de propriétés (e.g. convexe) était ensuite optimisée. Les propriétés de cette dernière nous permettait de conclure que le résultat ne sera pas trop mauvais. Un exemple de loss est la *logistic loss* définie comme&nbsp;:

$$\text{log}\big(1+e^{-z}\big),$$

où $z$ est justement le score retourné par notre modèle. Nous allons dans cette séquence une approche parallèle. Cette fois-ci, notre modèle ne donnera pas un score sur $\mathbb{R}$ mais sur $[0, 1]$ (la probabilité de la classe $1$ dans le cas à deux classes). Nous avons vu dans la séquence sur les fonctions proxy que considérer un modèle qui retourne un score sur $\mathbb{R}$ suivi de la *logistic loss* était équivalent au même modèle dont on donne le score à une fonction de lien sigmoid (qui retourne une probabilité) et qu'on optimise avec l'inverse de la log-vraisemblance&nbsp;: la vision statistique ou *machine learning* sont en réalité les deux faces d'une même pièce.

L'approche *machine learning* nous permet de réfléchir avec une vision purement prédictive et performances de prédiction alors que l'approche statistiques nous donne cette information sur les probabilités. Cette dernière peut être importante par exemple si on veut seuiller, e.g. je ne retourne la classe $1$ que si sa probabilité est supérieure à $99\%$. On verra que cette idée est clé lorsqu'on cherche à prédire des ensembles (cf. la séquence dédiée). 

Nous allons ici voir comment estimer ses probabilités sur un jeu d'apprentissage puis comment corriger/calibrer ces dernières pour qu'elles soient le plus réalistes possibles.

## II. Estimation de la probabilité conditionnelle

### A. *Proper loss* et *cross-entropy*

Nous allons ici prendre la perspective statistique, c'est-à-dire celle où on cherche à estimer la probabilité conditionnelle. Nous voulons construire un estimateur&nbsp;:

$$\hat{\eta}(x)=\hat{\mathbb{P}}\big(Y=1|X=x\big),$$

où $\hat{\eta}\in\mathcal{F}$ et $\mathcal{F}$ est notre ensemble de "probabilités conditionnelles" (e.g. les paramétrisations d'une régression logistique ou d'un réseau de neurones). Il existe de nombreuses stratégies afin d'atteindre cet objectif. L'une consiste à trouver une loss qui nous permettra de faire le meilleur choix de probabilité conditionnelle. La stratégie est souvent de maximiser la vraisemblance, ou, de manière totalement équivalente, de minimiser l'opposé de la log-vraisemblance&nbsp;:

$$\mathcal{L}(\hat{\eta})=-\sum_{i=1}^n Y_i\text{log}(\hat{\eta}(X_i))+(1-Y_i)\text{log}(1-\hat{\eta}(X_i)).$$

En espérance, nous notons&nbsp;:

$$\mathcal{L}_\eta(\hat{\eta})=-\mathbb{E}_X\big[\eta(X)\text{log}(\hat{\eta}(X))+(1-\eta(X))\text{log}(1-\hat{\eta}(X))\big].$$

Est-ce une bonne stratégie. Finalement la première chose que notre *loss* doit satisfaire est la suivante. Si on lui donnait la vraie probabilité conditionnelle $\eta$, serait-elle minimisée ? Serait-ce l'unique minimiseur ? Les définitions suivantes formalisent ces idées.

---

**Définition 1 (*Proper loss*)**

Une loss $\mathcal{L}_\eta$ est dite *proper* si $\eta$ est un minimiseur de cette dernière&nbsp;

$$\forall \hat{\eta},\ \mathcal{L}_\eta(\hat{\eta})\geq \mathcal{L}_\eta(\eta).$$

---

**Définition 2 (*Strictly proper loss*)**
Une loss $\mathcal{L}_\eta$ est dite *strictly proper* si $\eta$ est l'unique minimiseur de cette dernière&nbsp;

$$\forall \hat{\eta},\ \hat{\eta}\neq \eta,\ \mathcal{L}_\eta(\hat{\eta})> \mathcal{L}_\eta(\eta).$$

---

Il se trouve que la *cross-entropy* ou log-vraisemblance négative est *strictly proper*.

### B. Consistence de la *cross-entropy*


**Définition 3 (Plugin rule)**
Étant donnée un estimateur des probabilités conditionnelles $\hat{\eta}$, la plugin rule est celle qui retourne la classe associée à la plus forte probabilité&nbsp;:

$$g_{\hat{\eta}}(x)=\begin{cases}1\text{ si }\hat{\eta}(x)\geq 0.5\\ 0\text{ sinon.}\end{cases}$$

**<span style='color:blue'> Exercice</span>** 
Soit $\mathcal{F}$ un ensemble d'estimateurs du vrai $\eta$ (e.g. régression logistique) et $\mathcal{H}_{\mathcal{F}}$ les classifieurs (i.e. plugin rule) associés. Le choix du bon estimateur $\hat{\eta}\in\mathcal{F}$ se fait via la *cross entropy* qui est, rappelons le, *strictly proper*. Le classifieur associé est noté $g_{\hat{\eta}}\in\mathcal{H}_{\mathcal{F}}$. Notons $g^\star\in\mathcal{H}_{\mathcal{F}}$ le minimiseur du risque (ou le minimiseur du risque empirique après avoir vu une infinité de données). Le choix de $\hat{\eta}$ en minimisant la *cross entropy* sur une infinité de données garantit-il que le classifieur associé converge vers $g^\star\in\mathcal{H}_{\mathcal{F}}$ ?



 ----
**<span style='color:green'> Indices</span>** 
Considérer le scénario suivant. $\mathcal{X}=\{0, 1\}$, $\mu(0)=0.5$, $\mu(1)=0.5$, $\eta(0)=0.55$ et $\eta(1)=0.45$. Considérez maintenant la classe d'estimateurs suivante $\mathcal{F}=\{\eta_1,\eta_2\}$ où $\eta_1(0)=0.99$ et $\eta_1(1)=0.01$ ainsi que $\eta_2(0)=0.49$ et $\eta_1(1)=0.51$. Calculez la *cross-entropy* et déduisez-en le meilleur estimateur de la probabilité conditionnelle. Ensuite calculez le minimiseur du risque (l'estimateur dont la plugin rule fera le moins d'erreurs).



 ----

---

En réalité, la cross-entropy garantit que l'on obtienne la meilleure plugin-rule à partir du moment où on accepte certaines hypothèses. Ainsi, si le vrai $\eta$ fait partie des paramétrisations possibles alors, ça sera le cas.

## III. Calibration de la probabilité

Dans de nombreuses applications, il est nécessaire d'avoir une estimation "relativement correcte" de la probabilité conditionnelle $\eta$. Considérons le jeu de données synthétique suivant. Nous allons délibérément considérer un modèle avec une forte capacité de sur-apprentissage afin de mettre ces effets en avant dans des temps d'apprentissage raisonables.

In [ ]:
import numpy as np
from scipy.special import expit as sigmoid

np.random.seed(15)

def sample(n=100, d=10):
    beta = np.array([3. for _ in range(d)])
    X = np.random.uniform(-4, 4, size=(n, d))
    logits = np.dot(X, beta)
    y = np.random.binomial(1, p=sigmoid(logits))
    return X, y

In [ ]:
X, y = sample()

Entraînons notre premier modèle sur ces données via un réseau de neurones sur $\texttt{pytorch}$.

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn as nn
import torch
torch.manual_seed(42)

In [ ]:
def fullyconnected_layer(in_f, out_f):
    """
    " this function returns a fully connected layer with batchnorm
    """
    return nn.Sequential(
        nn.Linear(in_f, out_f),
        nn.BatchNorm1d(out_f),
        nn.ReLU()
    )

class Net(nn.Module):
    def __init__(self, n_labels=2, n_input=10, architecture=(2,)):
        """
        " n_labels is the dimension of the output
        " n_input is the dimension of the input
        " architecture describes the series of hidden layers
        """
        super(Net, self).__init__()

        layer_size = [n_input] + [i for i in architecture]

        layers = [
            fullyconnected_layer(in_f, out_f) for in_f, out_f in zip(layer_size, layer_size[1:])
        ]
        self.layers = nn.Sequential(*layers)
        self.fc = nn.Linear(architecture[-1], n_labels)

    def forward(self, x):
        x = self.layers(x)
        x = self.fc(x)
        return x

In [ ]:
class SyntheticDataset(Dataset):
    def __init__(self, X, y):
        self.dataset, self.label = X, y

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return torch.from_numpy(self.dataset[idx]).float(), int(self.label[idx])
    
dataset = SyntheticDataset(X, y)

In [ ]:
# TODO Décommenter les prints de loss, etc.
import matplotlib.pyplot as plt

def train_and_plot(lr=0.005, batch_size=50, epochs=5000, logs=10):
    model = Net(architecture=tuple(10 for _ in range(10)))
    # model.cuda()
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000)
    criterion = nn.CrossEntropyLoss()
    
    train_loader = DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=0)
    
    loss_history = []
    running_loss = 0.0
    for e in range(epochs):
        for idx, data in enumerate(train_loader):
            inputs, labels = data
            # labels = labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            running_loss += loss.item()
            if idx % logs == logs - 1:  # print every 2000 mini-batches
                print('\r[%d, %5d] loss: %.3f' % (e + 1, idx + 1, running_loss / logs), end="")
                loss_history.append(running_loss / logs)
                running_loss = 0.0

            loss.backward() # on calcule le gradient
            optimizer.step() # on fait un pas d'optimisation
        scheduler.step()
    print('\r************* Training done! *************')
    plt.figure(figsize=(12, 8))
    plt.plot(loss_history)
    plt.title('Loss')
    plt.show()
    return model

In [ ]:
model = train_and_plot(lr=0.005, batch_size=50, epochs=5000, logs=1)

L'optimisation se passe bien. Notre *loss* décroît et se rapproche asymptotiquement de $0$. On observe également quelques fluctuations liées au fait qu'on optimise notre réseau par *batch* (i.e. SGD). Considérons les notations suivantes. Soit $h:\mathcal{X}\rightarrow\mathbb{R}$ notre réseau de neurones dont la sortie sont appelés *logit*. Notons $\sigma(z)=(1+e^{-z})^{-1}$ la fonction sigmoid. C'est la fonction de lien qui permet de transformer nos *logits* en probabilité. Nous avons donc un estimateur de la vraie probabilité conditionnelle&nbsp;:

$$\hat{\eta}(x)=\sigma(h(x)).$$

Notre réseau est optimisé via la *cross-entropy* décrite plus haut&nbsp;:

$$\mathcal{L}(h)=-\frac{1}{n}\sum_{i=1}^n y_i\log\big(\sigma(h(x_i))\big)+(1-y_i)\log\big(1-\sigma(h(x_i))\big).$$

**<span style='color:blue'> Question</span>** 
Notre *loss* atteint quasiment $0$. Supposons qu'elle soit aussi proche de $0$ que l'on veut. Que pouvons-nous dire sur l'erreur de classification $0/1$&nbsp;:

$$Re(h)=\frac{1}{n}\sum_{i=1}^n\textbf{1}\{h(x_i)\neq y_i\},$$

relativement à la *plugin-rule* associée à $\sigma(h(x))$&nbsp;?



 ----

Si notre *loss* est proche de $0$, alors les probabilités retournées par notre modèle sont soit très proches de $0$ pour les points de notre jeu de données de la classe $0$, soit très proches de $1$ pour les points associés à la classe $1$. Notre modèle indique donc une "forte confiance" dans ses prédictions tout en ne faisant que peu d'erreurs sur notre jeu d'apprentissage. Il existe une stratégie permettant de quantifier la qualité de la "confiance" d'un modèle&nbsp;: la courbe de calibration. Cette dernière va confronter les scores de prédiction de notre modèle à leur valeur empirique sur nos données (i.e. si notre modèle prédit $70\%$, on s'attend que $70\%$ des points associés à ce score soient de la classe $1$).

In [ ]:
trainloader = DataLoader(dataset, shuffle=True, batch_size=100, num_workers=0)

In [ ]:
def loss_value(loader, temperature=None):
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        model.eval()
        running_loss = 0
        true_labels = []
        pred = []
        for idx, data in enumerate(loader):
            inputs, labels = data
            true_labels.append(labels.cpu().detach().numpy())
            outputs = model(inputs)
            if temperature is not None:
                outputs = outputs * temperature
            pred.append(torch.softmax(outputs, axis=1)[:, 1].cpu().detach().numpy())
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    true_labels = np.concatenate(true_labels, axis=0)
    pred = np.concatenate(pred, axis=0)
    return true_labels, pred, running_loss

true_train, pred_train, loss_train = loss_value(trainloader)
print('Cross-entropy sur le train: %.3f' % loss_train)

In [ ]:
from sklearn.calibration import calibration_curve
def calibration(true_value, pred_value):
    fig = plt.figure(figsize=(12, 8))
    ax1 = fig.gca()
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

    fraction_of_positives, mean_predicted_value = \
        calibration_curve(true_value, pred_value, n_bins=10)

    ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
             label="%s" % ('Deep net', ))

    ax1.set_ylabel("Fraction of positives")
    ax1.set_xlabel("Mean predicted value")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots')
    plt.show()
    
calibration(true_train, pred_train)

Ce résultat était attendu. Notre modèle est en surapprentissage : il ne se trompe pas sur le jeu d'apprentissage et prédit des scores indiquant une forte confiance. Nous devons évaluer la calibration **et** la *cross-entropy* sur un jeu de test afin d'avoir une idée de la qualité des prédictions de notre modèle.

In [ ]:
X_test, y_test = sample(n=500)
testset = SyntheticDataset(X_test, y_test)
testloader = DataLoader(testset, shuffle=True, batch_size=500, num_workers=0)

In [ ]:
true_test, pred_test, loss_test = loss_value(testloader)
print('Cross-entropy sur le train: %.3f' % loss_test)
calibration(true_test, pred_test)

Notons tout d'abord qu'un modèle qui prédirait $50\%$ (une sorte de modèle aléatoire) pour chacun des points de notre jeu d'apprentissage aurait une *cross-entropy* de $0.7$. Notre modèle est donc pire, d'un point de vue des probabilités, qu'un modèle aléatoire.

**<span style='color:blue'> Question</span>** 
Une forte *cross-entropy* en validation indique-t-elle un mauvais classifieur du point de vue de l'erreur $0/1$&nbsp;?



 ----

Concernant notre courbe de calibration, le meilleur modèle (du point de vue de la calibration) est celui qui serait situé totalement sur la diagonal. C'est le modèle dont les probabilités estimées correspond aux erreurs de prédictions constatés empiriquement. On observe que les probabilités estimées ne correspondent pas du tout à ce qui est observé en pratique. Ainsi, lorsqu'on regarde les points prédit à $45\%$ comme appartenant à la classe $1$, ils sont en réalité à $0\%$ du côté de la classe $1$. À l'inverse, si je prends ceux qui sont à $20\%$ associés à la classe $1$, ils le sont réellement $65\%$ du temps. Nos probabilités sont absolument impossibles à interpréter.


**<span style='color:blue'> Question</span>** 
Un modèle situé parfaitement sur la diagonale de calibration est-il nécessairement un bon modèle prédictif de classification&nbsp;?



 ----

Notre stratégie va être de trouver une "adaptation" de notre modèle qui ne change en rien son pouvoir prédictif mais qui calibre mieux les probabilités. Observons tout d'abord notre fonction de lien.

In [ ]:
x = np.linspace(-5, 5, 100)
y = sigmoid(x)
plt.figure(figsize=(12, 8))
plt.plot(x, y)
plt.show()

Lorsque les valeurs des *logits* (i.e. $h(x)$) s'écartent de $0$, la probabilité estimée se rapproche des fortement de $0$ ou de $1$. Ainsi, en réduisant l'amplitude des *logits*, nous rapprochons nos probabilités estimées de $50\%$. Cela nous permet de réduire l'excès de confiance de notre modèle. On constate de plus que cela ne change en rien le pouvoir prédictif de notre modèle. Illustrons ce dernier point par un exemple.

In [ ]:
x = np.random.uniform(-50, 50, (10, 1))

y = np.concatenate([
    sigmoid(x),
    sigmoid(x/50) # on divise nos logits par 50
], axis=1)
print(y)
print(y>0.5) # on a la valeur True lorque c'est la classe $1$ qui est prédite

On remarque qu'indépendamment de l'amplitude de nos logits, les prédictions restent les mêmes. Cela se généralise bien sûr au cas multiclasse.

Nous devons donc estimer un facteur multiplicatif permettant d'obtenir une meilleure calibration (i.e. on veut être sur la diagonale de calibration). Ce n'est en réalité pas suffisant car il suffit que le facteur multiplicatif soit aussi proche de $0$ qu'on le souhtaite afin d'être aussi proche de la diagonale que possible. La stratégie va être d'optimiser ce score multiplicatif afin que notre modèle soit bon sur un ensemble de validation. Ainsi, on ne change pas la forme de notre modèle prédictif (la frontière de décision reste fixe), mais on altère l'estimation des probabilités afin d'obtenir un bon score sur un jeu de validation. Notre modèle devient donc&nbsp;:

$$\sigma(t h(x)),$$
où $t\in\mathbb{R}^+$ est notre scalaire qu'on appelle "température". L'idée va ensuite d'optimiser notre modèle dont l'unique paramètre est $t$ pour $h$ fixé sur un jeu d'apprentissage avec la *cross-entropy*.

In [ ]:
X_val, y_val = sample(n=500)
valset = SyntheticDataset(X_val, y_val)
valloader = DataLoader(valset, shuffle=True, batch_size=500, num_workers=0)

**<span style='color:blue'> Exercice</span>** 
Complétez le code ci-dessous afin d'estimer le scalaire de température.



 ----

In [ ]:
temperature = nn.Parameter(torch.ones(1))
####### Complete this part ######## or die ####################
...
...
...
###############################################################
print('\nTemperature:', temperature.detach())


In [ ]:
true_test, pred_test, loss_test = loss_value(testloader, temperature)
print('Cross-entropy sur le test: %.3f' % loss_test)


Notre modèle est donc meilleur qu'un modèle aléatoire en terme d'estimation des probabilités sur un jeu de test.

In [ ]:
calibration(true_test, pred_test)


On observe également que la calibration est bien meilleure !